In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

In [6]:
Root_dir = './data/'
Train_dir = Root_dir + 'train.csv'
aug_root = Root_dir + 'new_data/'

In [10]:
X_new = []
Y_new = []
file_list = os.listdir(aug_root)
for i in range(10):
    image_list = os.listdir(aug_root + file_list[i])
    for image in image_list:
        new_image = cv2.imread(aug_root + file_list[i] + '/' +image, cv2.IMREAD_GRAYSCALE)
        new_image = np.expand_dims(new_image, axis = -1)
        X_new.append(new_image)
        Y_new.append(i)
X_new_data = np.array(X_new)
Y_new_data = np.expand_dims(np.array(Y_new), axis = -1)

In [12]:
origin_data = pd.read_csv(Train_dir).set_index('id')
X_origin = origin_data.iloc[:,2:].values.reshape(-1,28,28,1)
X_origin = X_origin/255.0
Y_origin = np.expand_dims(origin_data['digit'].values, axis= -1)

In [14]:
X_data = np.concatenate((X_origin, X_new_data), axis = 0)
Y_data = np.concatenate((Y_origin, Y_new_data), axis = 0)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_validation, Y_train, Y_validation = train_test_split(X_data, Y_data, test_size=0.1, random_state=321)

In [16]:
def create_module(layer, channel_size, kernel_size):
    conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(layer)
    batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
    conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(batch_layer)
    batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
    pooling_layer = tf.keras.layers.MaxPooling2D((2, 2))(batch_layer)
    return pooling_layer

def create_model():
    inputs = tf.keras.layers.Input(shape = (X_train.shape[1:]))
    module_1 = create_module(inputs, 64, 2)
    module_2 = create_module(module_1, 64, 2)
    flatten_layer = tf.keras.layers.Flatten()(module_2)
    outputs=tf.keras.layers.Dense(10, activation='softmax')(flatten_layer)

    return tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [17]:
model=create_model()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model_path=Root_dir + 'model_orthogonal/'

if not os.path.exists(model_path):
  os.mkdir(model_path)

checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path+'Val_{val_accuracy:.3f}.h5',
    monitor='val_accuracy', 
    save_best_only=True
)

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.1,
    patience=10,
    mode='max'
)

model.fit(
    X_train, Y_train,
    epochs=100, batch_size=8,
    validation_data=(X_validation, Y_validation), shuffle=True, 
    callbacks=[checkpoint, early_stopping]
)

NameError: name 'y_train' is not defined